In [1]:
from src.pdfinsight import pdf_extractor, remove_toc, combine_categories, pivot_df_by_heading
import pandas as pd
import os

In [2]:
df_combined = pd.DataFrame()
for (root,dirs,files) in os.walk('.', topdown=True):
    for file in files:
        # only use it on pdf docs. ignore other file type such as txt
        if file.split(".")[-1] == "pdf":
            # print(root+"\\"+file)
            df = pdf_extractor(root+"\\"+file, toc_pages = 1, precision_dp = 2)
            df_combined = pd.concat([df_combined,df])
    break

# reset index
df_combined.reset_index(drop=True, inplace=True)

In [3]:
df_combined

,file,page,block,refined_block,block_ymin_diff,block_is_list,xmin,ymin,xmax,ymax,...,text_rep,xmin_rep,ymin_rep,font_size,font_characteristics,font,font_color,text,image,cat
0,01_CPM111_Preparing Cost Estimates (Constructi...,1,3,1,NaN,False,204.119781,107.255196,442.108307,121.291252,...,1,1,1,14.04,20,Times-Bold,0,SAFETY AND CONTRACTS GROUP,,toc
1,01_CPM111_Preparing Cost Estimates (Constructi...,1,3,1,NaN,False,133.678894,149.989777,514.405884,171.943481,...,1,1,1,21.95,20,Times-Bold,0,CONTRACT PROCEDURE MANUAL,,toc
2,01_CPM111_Preparing Cost Estimates (Constructi...,1,3,1,NaN,False,259.439178,173.975174,386.667450,188.011215,...,1,1,1,14.04,20,Times-Bold,0,(CONSTRUCTION),,toc
3,01_CPM111_Preparing Cost Estimates (Constructi...,1,3,1,NaN,False,186.238617,243.642517,460.856415,261.637390,...,1,1,1,17.99,20,Times-Bold,0,PREPARING COST ESTIMATES,,toc
4,01_CPM111_Preparing Cost Estimates (Constructi...,1,3,1,NaN,False,204.599152,349.362640,442.616943,367.357513,...,1,1,1,17.99,20,Times-Bold,0,Document No.: CPM/C/M/111,,toc
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3907,220 Performance Report (Construction).pdf,6,8,73,30.0,True,125.279114,148.928619,453.813110,160.925079,...,1,70,2,12.00,4,Times-Roman,0,contractor’s/consultant’s performance with the...,,content
3908,220 Performance Report (Construction).pdf,6,8,73,30.0,True,458.279327,148.928619,464.277557,160.925079,...,2,1,2,12.00,4,Times-Roman,0,.,,content
3909,220 Performance Report (Construction).pdf,6,11,74,614.0,False,89.879158,763.199951,93.117302,769.678101,...,2,11,2,6.48,4,Times-Roman,0,4,,footnote
3910,220 Performance Report (Construction).pdf,6,11,74,614.0,False,93.119171,764.038208,555.447998,773.995178,...,1,4,3,9.96,4,Times-Roman,0,BCA’s Advisory Note on ‘Review of Performance ...,,footnote


In [4]:
# we check the ymin difference between each row INSTEAD OF BLOCK. 
# this is important in determining the para_thres
# text below the para_thres would be considered to be in the same block of text (ie para) 
# seems like if those between 0 to 20 is above 70%, we can set that as para_thres
pd.DataFrame(pd.cut([idx for idx in df['ymin_round'].diff()], [-1000, -0.01,10, 20, 30, 40, 50, 1000])).value_counts(normalize=True).sort_index()

(-1000.0, -0.01]    0.019157
(-0.01, 10.0]       0.452107
(10.0, 20.0]        0.325670
(20.0, 30.0]        0.126437
(30.0, 40.0]        0.026820
(40.0, 50.0]        0.019157
(50.0, 1000.0]      0.030651
Name: proportion, dtype: float64

In [5]:
# double check the ymin_thres between REFINED_BLOCKS
# as long as our defined para_thres of 20 is the majority
# then should be safe to use that as para_thres
pd.DataFrame(pd.cut([idx for idx in df['block_ymin_diff']], [-1000, -0.01,10, 20, 30, 40, 50, 1000])).value_counts(normalize=True).sort_index()

(-1000.0, -0.01]    0.093385
(-0.01, 10.0]       0.000000
(10.0, 20.0]        0.431907
(20.0, 30.0]        0.276265
(30.0, 40.0]        0.112840
(40.0, 50.0]        0.027237
(50.0, 1000.0]      0.058366
Name: proportion, dtype: float64

In [6]:
df_combined = remove_toc(df_combined)
df_combined

,file,page,block,refined_block,block_ymin_diff,block_is_list,xmin,ymin,xmax,ymax,...,text_rep,xmin_rep,ymin_rep,font_size,font_characteristics,font,font_color,text,image,cat
0,01_CPM111_Preparing Cost Estimates (Constructi...,2,2,5,-689.0,False,231.359161,45.498444,293.847656,55.455395,...,4,8,8,9.96,4,Times-Roman,0,CPM/C/M/111,,header
1,01_CPM111_Preparing Cost Estimates (Constructi...,2,2,5,-689.0,False,478.919342,45.498444,529.767639,55.455395,...,4,8,8,9.96,4,Times-Roman,0,Revision : 1,,header
2,01_CPM111_Preparing Cost Estimates (Constructi...,2,2,5,-689.0,False,231.359161,57.249428,355.677551,69.245895,...,4,8,4,12.00,4,Times-Roman,0,Preparing Cost Estimates,,header
3,01_CPM111_Preparing Cost Estimates (Constructi...,2,2,5,-689.0,False,478.919342,57.018463,526.647766,66.975410,...,1,8,4,9.96,4,Times-Roman,0,Page 1 of 4,,page_number
4,01_CPM111_Preparing Cost Estimates (Constructi...,2,6,6,39.0,False,95.039169,95.627869,154.077438,107.624336,...,1,6,1,12.00,20,Times-Bold,0,PURPOSE,,content
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3409,220 Performance Report (Construction).pdf,6,8,73,30.0,True,125.279114,148.928619,453.813110,160.925079,...,1,70,2,12.00,4,Times-Roman,0,contractor’s/consultant’s performance with the...,,content
3410,220 Performance Report (Construction).pdf,6,8,73,30.0,True,458.279327,148.928619,464.277557,160.925079,...,2,1,2,12.00,4,Times-Roman,0,.,,content
3411,220 Performance Report (Construction).pdf,6,11,74,614.0,False,89.879158,763.199951,93.117302,769.678101,...,2,11,2,6.48,4,Times-Roman,0,4,,footnote
3412,220 Performance Report (Construction).pdf,6,11,74,614.0,False,93.119171,764.038208,555.447998,773.995178,...,1,4,3,9.96,4,Times-Roman,0,BCA’s Advisory Note on ‘Review of Performance ...,,footnote


In [10]:
transformed_df = pivot_df_by_heading(df_combined)

In [11]:
transformed_df

,file,heading1,heading2,heading3,content
0,01_CPM111_Preparing Cost Estimates (Constructi...,PROCEDURE,1. Uses of Cost Estimate,None,1.1 A cost estimate may be required for variou...
1,01_CPM111_Preparing Cost Estimates (Constructi...,PROCEDURE,2. Method of Estimating,None,"2.1 In arriving at the estimate, Contracts wou..."
2,01_CPM111_Preparing Cost Estimates (Constructi...,PROCEDURE,3. Format for Presentation of Estimates,None,3.1 Refer to User Guide for guidelines on the ...
3,01_CPM111_Preparing Cost Estimates (Constructi...,PROCEDURE,4. Revision of Estimate,None,4.1 Contracts shall revise the original estima...
4,01_CPM111_Preparing Cost Estimates (Constructi...,PROCEDURE,5. Review of Budget Estimates,None,1 5.1 Budget estimates for development project...
...,...,...,...,...,...
97,217 Amicable and Commercial Settlement (Constr...,None,COMMERCIAL SETTLEMENT\nPROCEDURE\n4.,7. Decision,7.1 The Commercial Settlement offer proposed b...
98,217 Amicable and Commercial Settlement (Constr...,None,Amount Approving Authority,8. CONFIDENTIALITY,8.1 All records relating to Commercial Settlem...
99,220 Performance Report (Construction).pdf,PROCEDURE,1. General,None,"1.1 Performance reports on contractors, consul..."
100,220 Performance Report (Construction).pdf,PROCEDURE,2. Performance Assessment and Reports Submission,None,2.1 Contractors’ performance report shall be p...


In [12]:
transformed_df.loc[101,'content']

'3.1 For BCA registered contractors, the performance report shall be distributed as follows:\na) Original – BCA\nb) Copy – Engineer\nc) Copy – CA\nd) Copy – T&P\n3.2 For design consultants, the performance report shall be distributed as follows:\na) Original – BCA (via BCA CPAS)\nb) Copy – Engineer\nc) Copy – CA\n3.3 For other consultants and non-BCA registered contractors, the performance report shall be distributed as follows:\na) Original – 1 Contracts (Civil 1) Division\nPreparing Performance Report\nb) Copy – Engineer\nc) Copy – CA\n3.4 On the completion of the contract, the Engineer is to share the details of 4 contractor’s/consultant’s performance with the contractor/consultant .'